# **Modelado de Datos para Machine Learning**

<div style='text-align: justify'>
En esta sección, daremos inicio al proceso de modelado de datos para nuestro modelo de machine learning. Utilizaremos los resultados obtenidos de los procesos de Extracción, Transformación y Carga (ETL), así como del Análisis Exploratorio de Datos (EDA) desarrollados hasta el momento. A través de este proceso, prepararemos nuestros datos para entrenar y evaluar modelos de machine learning con el objetivo de obtener predicciones precisas y útiles.
</div>

## 1. Importamos las librerias

In [1]:
import pandas as pd
import numpy as np

import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator

import fastparquet as fp 
import pyarrow as pa
import pyarrow.parquet as pq
import funciones_basicas as fb

### Modelo de Recomendación basado en Elementos

<div style='text-align: justify'>
Un sistema de recomendación basado en elementos, específicamente un modelo de recomendación de ítem a ítem, utiliza la métrica de la similitud del coseno para sugerir elementos similares a aquellos que un usuario ha mostrado interés previamente o ha interactuado. Este enfoque se apoya en la premisa de que si a un usuario le agradó un cierto elemento, es probable que también le gusten elementos similares. La caracterización de un elemento se suele representar mediante un vector que contiene información pertinente sobre dicho elemento. Por ejemplo, en un sistema de recomendación de libros, el perfil de un libro podría incluir detalles como el género, el autor, la trama, las etiquetas asociadas, entre otros. La similitud del coseno se calcula para evaluar la similitud entre los perfiles de los elementos y se fundamenta en el ángulo entre los vectores de características de los elementos. Cuanto menor sea el ángulo entre los vectores, mayor será la similitud del coseno y más parecidos serán los elementos.
</div>

## 1. Cargamos los datos

In [2]:
modelo_item = fb.cargar_datos_parquet('PARQUET\MODELO_ITEM.parquet')
modelo_item.head(3)

,ID_GAME,NOMBRE_JUEGO,GENEROS
0,761140,LOST SUMMONER KITTY,ACTION
1,643980,IRONBOUND,FREE TO PLAY
2,670290,REAL POOL 3D - POOLIANS,CASUAL


### 2. Vamos a proceder a crear variables dummy para nuestro modelo. 

<div style='text-align: justify'>
Las variables dummy son variables binarias que representan categorías distintas en un conjunto de datos. Estas variables son útiles cuando queremos incluir información categórica en un modelo de machine learning que requiere entradas numéricas. Al convertir las categorías en variables binarias, podemos capturar la información de forma que el modelo pueda interpretarla adecuadamente durante el proceso de entrenamiento y predicción.
</div>

In [3]:
#Creamos los dummies
modelo_item = pd.get_dummies(modelo_item, columns=["GENEROS"], prefix="")

modelo_item = modelo_item.groupby(["ID_GAME","NOMBRE_JUEGO"]).sum().reset_index()

modelo_item.head(4)

,ID_GAME,NOMBRE_JUEGO,_ACCOUNTING,_ACTION,_ADVENTURE,_ANIMATION &AMP; MODELING,_AUDIO PRODUCTION,_CASUAL,_DESIGN &AMP; ILLUSTRATION,_EARLY ACCESS,...,_PHOTO EDITING,_RACING,_RPG,_SIMULATION,_SOFTWARE TRAINING,_SPORTS,_STRATEGY,_UTILITIES,_VIDEO PRODUCTION,_WEB PUBLISHING
0,10,COUNTER-STRIKE,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1002,RAG DOLL KUNG FU,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100400,SILO 2,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10090,CALL OF DUTY: WORLD AT WAR,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3. Determinando la Coincidencia de Coseno

<div style='text-align: justify'>
Para encontrar elementos similares para recomendar, se calcula la concordancia de coseno entre los perfiles de todos los pares de elementos. La concordancia de coseno mide la similitud en un espacio multidimensional, variando de -1 (opuestos) a 1 (idénticos). Cuanto mayor sea la concordancia de coseno, más similares serán los elementos. Este enfoque identifica elementos con características compartidas, lo que facilita la recomendación de elementos relevantes para los usuarios.
</div>

In [4]:
similitudes = cosine_similarity(modelo_item.iloc[:,3:])

In [5]:
def recomendacion_juego(id):
    
    id = int(id)
    # Filtramos el juego e igualarlo a  su ID
    juego_seleccionado = modelo_item[modelo_item['ID_GAME'] == id]
    # devolvemos error en caso de vacio
    if juego_seleccionado.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    
    # Calculamos la matriz de similitud coseno
    #similitudes = cosine_similarity(modelo_item.iloc[:,3:])
    
    # Calculamos la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = similitudes[modelo_item[modelo_item['ID_GAME'] == id].index[0]]
    
    # Calculamos los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]
    
    # Obtenemos los nombres de los juegos 5 recomendados
    juegos_recomendados = modelo_item.iloc[indices_juegos_similares]['NOMBRE_JUEGO']
    
    return juegos_recomendados

## 4. Partimos el dataframe en 2 para poder montar el modelo en render

In [6]:
# Revisamos la cantidad de filas
cant_filas= len(modelo_item)

# Calculamos la mitad
mitad_filas= cant_filas // 10

# Seleccionamos la mitad superior
modelo_train= modelo_item.iloc[:mitad_filas]
modelo_train.shape

(3213, 25)

In [7]:
similitudes_train = cosine_similarity(modelo_train.iloc[:,3:])

## 5. Guardamos los datos el nuevo dataframe

In [8]:
fb.guardar_datos(modelo_train, 'PARQUET\TABLAS UNIDAS\MODELO_TRAIN.parquet')

## 6. Modelo ML

<div style='text-align: justify'>
Se crea una función especial para buscar solo juegos similares al juego proporcionado. Esta función ayuda a acelerar el proceso de renderizado al calcular las similitudes para un solo juego en lugar de para todos los juegos cada vez. La función toma un juego como entrada y mide su similitud con todos los demás juegos en función de sus características, lo que permite obtener recomendaciones personalizadas para un juego específico sin tener que calcular todas las similitudes cada vez.
</div>

In [9]:
modelo_train.head()

,ID_GAME,NOMBRE_JUEGO,_ACCOUNTING,_ACTION,_ADVENTURE,_ANIMATION &AMP; MODELING,_AUDIO PRODUCTION,_CASUAL,_DESIGN &AMP; ILLUSTRATION,_EARLY ACCESS,...,_PHOTO EDITING,_RACING,_RPG,_SIMULATION,_SOFTWARE TRAINING,_SPORTS,_STRATEGY,_UTILITIES,_VIDEO PRODUCTION,_WEB PUBLISHING
0,10,COUNTER-STRIKE,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1002,RAG DOLL KUNG FU,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100400,SILO 2,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10090,CALL OF DUTY: WORLD AT WAR,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,100980,3D-COAT V4.8,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
def encontrar_juegos_similares(id: str):
    juego_seleccionado = modelo_train[modelo_train['ID_GAME'] == id]
    if juego_seleccionado.empty:
        return f"El juego con el ID {id} especificado no existe en la base de datos."
    
    indice_juego = juego_seleccionado.index[0]
    similarity_scores = similitudes_train[indice_juego]
    
    # Verificamos si hay al menos 6 juegos en la base de datos
    if len(similarity_scores) < 6:
        return "No hay suficientes juegos en la base de datos para proporcionar recomendaciones."
    
    # Seleccionamos los índices de los juegos más similares
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]
    juegos_recomendados = modelo_train.iloc[indices_juegos_similares]['NOMBRE_JUEGO']
    
    return juegos_recomendados.tolist()


In [11]:
encontrar_juegos_similares('1002')

['FALLEN ENCHANTRESS: LEGENDARY HEROES - QUEST PACK DLC',
 'SPACE HULK - SWORD OF HALCYON CAMPAIGN',
 'SPACE HULK - KRAKEN SKIN DLC',
 'AVADON 2: THE CORRUPTION',
 'PRISON ARCHITECT']

## 7. Creamos una función para obtener recomendaciones a partir de los datos anteriores

In [12]:
modelo_train.head(2)

,ID_GAME,NOMBRE_JUEGO,_ACCOUNTING,_ACTION,_ADVENTURE,_ANIMATION &AMP; MODELING,_AUDIO PRODUCTION,_CASUAL,_DESIGN &AMP; ILLUSTRATION,_EARLY ACCESS,...,_PHOTO EDITING,_RACING,_RPG,_SIMULATION,_SOFTWARE TRAINING,_SPORTS,_STRATEGY,_UTILITIES,_VIDEO PRODUCTION,_WEB PUBLISHING
0,10,COUNTER-STRIKE,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1002,RAG DOLL KUNG FU,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
def recomendacion_juego(id: str):
    
    # Verificamos si el juego con game_id existe en df_games
    game = modelo_train[modelo_train['ID_GAME'] == id]

    if game.empty:
        return("El juego '{id}' no posee registros.")
    
    # Obtenemos el índice del juego dado
    idx = game.index[0]

    # Tomamos una muestra aleatoria del DataFrame df_games
    sample_size = 2000  # Definimos el tamaño de la muestra (ajusta según sea necesario)
    df_sample = modelo_train.sample(n=sample_size, random_state=42)  # Ajustamos la semilla aleatoria según sea necesario

    # Calculamos la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([modelo_train.iloc[idx, 3:]], df_sample.iloc[:, 3:])

    # Obtenemos las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordenamos los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtenemos los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Listamos los juegos similares (solo nombres)
    similar_game_names = df_sample['NOMBRE_JUEGO'].iloc[similar_game_indices].tolist()

    return {"JUEGOS SIMILARES": similar_game_names}

In [14]:
recomendacion_juego('1002')

{'JUEGOS SIMILARES': ['THE CLOCKWORK MAN',
  "TALISMAN - CHARACTER PACK #3 - DEVIL'S MINION",
  'PLANET BUSTERS',
  'FALLEN ENCHANTRESS: LEGENDARY HEROES - LOOT PACK DLC',
  'LORDS OF FOOTBALL: UNITED STATES']}